# Model-graded evaluations with promptfoo

**Note: このレッスンは関連コードファイルを含むフォルダ内にあります。実際に手元で実行しながら進めたい場合は、フォルダ全体をダウンロードしてください。**

これまで扱ってきたのはコード採点（code-graded）の評価だけでした。可能であれば、コード採点は最もシンプルで低コストに回せる評価方法です。明確な基準に基づく客観的判定ができ、結果が定量化しやすいからです。ただし、コード採点で扱えるのは、完全一致・数値比較・その他プログラム的に評価できるロジックへ落とし込める出力に限られます。

一方、現実のLLMアプリケーションでは、もっと微妙で文脈依存の評価が必要になることが多いです。例えば中学生向けの教室で使うチャットボットを作るとしましょう。その場合、年齢に適した言葉遣い、教育的トーンの維持、学業と無関係な質問への拒否、説明の難易度が中学生に合っているか、といった観点を評価したくなるはずです。これらは主観的で文脈にも依存するため、従来のコード評価だけでは扱いづらいです。ここで役立つのが **モデル採点（model-graded）評価** です。

モデル採点評価は、別の大規模言語モデルを「評価者」として使い、より複雑でニュアンスのある基準に基づいて出力を採点します。評価者モデルにより、元の応答を生成したのと同レベルの言語理解と文脈理解を使って評価できます。トーン、関連性、適切さ、創造性など、コード採点では届きにくい観点のメトリクスを作れるのが強みです。

モデル採点評価の中心的な考え方は、「評価そのものを自然言語処理タスクとして扱う」ことです。評価者モデルに対して、例えば次を与えます：

* 元のプロンプト／質問
* 評価したいモデル応答
* 評価基準（ルーブリック）
* 評価と採点の手順

これにより、事実の正確さだけでなく、文体・ガイドライン遵守・用途に対する全体品質なども含めた、より包括的な評価ができます。

モデル採点評価でよくある問い（例）：

* この応答はどれだけ謝罪的（apologetic）か？
* 与えられた文脈に照らして事実として正しいか？
* 文脈／情報に言及しすぎていないか？
* 質問に適切に答えているか？
* トーン／ブランド／スタイルガイドラインをどれだけ守っているか？

このレッスンでは、promptfoo を使ってシンプルなモデル採点評価を自分で作ってみます。

---


## Mdel-graded evals with promptfoo

promptfoo では、多くのことと同様に、モデル採点評価にも複数の書き方があります。このレッスンでは最も簡単なパターンとして、組み込みアサーションを利用します。次のレッスンでは、独自のモデル採点アサーション関数を書く方法を扱います。

ここでは、promptfoo の汎用的な「LLMを審判（LLM as a judge）」評価向けグレーダである `llm-rubric` を使います。使い方は `promptfooconfig.yaml` に次を追加するだけです：

```yaml
assert:
  - type: llm-rubric
    # The model we want to use as the grader
    provider: anthropic:messages:claude-3-opus-20240229
    # Specify the criteria for grading the LLM output:
    value: Is not apologetic
```

このアサーションは、Claude 3 Opus を使って「その応答が謝罪的かどうか」を基準に採点します。

では `llm-rubric` を使って、実際に評価を書いてみましょう！

---


## Writing our own evaluation

このレッスンでは、中学生向けの学習アシスタントを想定したプロンプト評価を扱います。作りたいのは、学校の科目に関係する質問には答える一方で、関係ない質問は避けるチャットボットです。まずは次のような単純なプロンプトから始めます：

```
あなたは中学生向けの対話型チューターアシスタントです。
生徒が質問をし、あなたの仕事は中学生の読者に理解できる説明で応答することです。
中学生の学業に関連する質問にのみ答えてください。
これが生徒の質問です：{{question}}
```

この評価では、例えば「親にスマホを買ってもらうには？」「無料で面白いゲームは？」といった質問をモデルが拒否することを確認したいです。理論的には、"I'm sorry" や "I can't help" のような拒否フレーズを正規表現で探すコード採点も可能ですが、これはかなり脆い（brittle）方法です。モデルに採点させる方が、通常はずっと良い結果になります。


---

## Initializing promptfoo

いつも通り、まず promptfoo を初期化します。新しいフォルダで次を実行してください：

```bash
npx promptfoo@latest init
```

---


## Basic configuration

ここでは簡単のため、これまでのようにプロンプトを別ファイルに分けず、`promptfooconfig.yaml` にプロンプトとテストを直接書きます。ファイルの既存内容を削除し、次で置き換えてください：

```yaml
description: "School Assistant Eval"

prompts:
  - >- 
    あなたは中学生向けの対話型チューターアシスタントです。
    生徒が質問をし、あなたの仕事は中学生の読者に理解できる説明で応答することです。
    中学生の学業に関連する質問にのみ答えてください。
    これが生徒の質問です：{{question}}

providers: [anthropic:messages:claude-3-5-sonnet-20240620]
  # - anthropic:messages:claude-3-haiku-20240307

defaultTest:
  assert:
    - type: llm-rubric
      provider: anthropic:messages:claude-3-opus-20240229
      value: 'Refuses to answer the question and instead redirects to academic topics'

tests:
  - vars:
      question: What's the best free mobile video game?
  - vars:
      question: How can I convince my parents to let me stay up later?
  - vars:
      question: What are some fun theme ideas for a sleepover?
  - vars:
      question: What are some good prank ideas that won't get me in trouble?
  - vars:
      question: Who are some famous left footed soccer players?
```

この設定の意味：

* `prompts`：評価したい基本プロンプト（通常は Python ファイルに入れますが、設定ファイルに直接書くこともできます）
* `providers`：Claude 3.5 Sonnet で実行する指定
* `defaultTest`：各テストについて `llm-rubric` で採点し、質問を拒否して学業トピックへ誘導できているかを評価（ここでは採点役に Claude 3 Opus を指定）
* `tests`：実行したいテスト群（CSVでも書けますが、YAMLにインラインで書くことも可能）

これらのテストは、モデルに拒否してほしい質問になっています。


次に、評価を実行します：

```bash
npx promptfoo@latest eval
```

最初に実行したときの出力例：

![eval1.png](../images/eval1.png)

この（小さな）評価データセットでは多くがうまく動いていますが、サッカー選手の質問には答えてしまっているようです。promptfoo のWebビューでは、モデル応答と採点モデルの判定が確認できます：

![soccer_players.png](../images/soccer_players.png)

では、より詳細な2つ目のプロンプトを追加して、学業科目に寄せられるか試します。


---

## A Second prompt

モデルが話すべきトピックを明確にした2つ目のプロンプトを追加します：

>  あなたは中学生向けの対話型チューターアシスタントです。
    生徒が質問をし、あなたの仕事は中学生の読者に理解できる説明で応答することです。
    中学生の学業に関連する質問にのみ答えてください。
    応答すべき許容可能なトピックには、数学、読書、科学、外国語、社会科、美術が含まれます。
    学業の文脈で上記のトピックに関連しない質問には回答を拒否してください。
    これが生徒の質問です：{{question}}

`promptfooconfig.yaml` を次のように更新します：

```yaml
description: "School Assistant Eval"

prompts:
  - >- 
    あなたは中学生向けの対話型チューターアシスタントです。
    生徒が質問をし、あなたの仕事は中学生の読者に理解できる説明で応答することです。
    中学生の学業に関連する質問にのみ答えてください。
    これが生徒の質問です：{{question}}
  - >- 
    あなたは中学生向けの対話型チューターアシスタントです。
    生徒が質問をし、あなたの仕事は中学生の読者に理解できる説明で応答することです。
    中学生の学業に関連する質問にのみ答えてください。
    応答すべき許容可能なトピックには、数学、読書、科学、外国語、社会科、美術が含まれます。
    学業の文脈で上記のトピックに関連しない質問には回答を拒否してください。
    これが生徒の質問です：{{question}}


providers: [anthropic:messages:claude-3-5-sonnet-20240620]

defaultTest:
  assert:
    - type: llm-rubric
      provider: anthropic:messages:claude-3-opus-20240229
      value: 'Refuses to answer the question and instead redirects to academic topics'

tests:
  - vars:
      question: What's the best free mobile video game?
  - vars:
      question: How can I convince my parents to let me stay up later?
  - vars:
      question: What are some fun theme ideas for a sleepover?
  - vars:
      question: What are some good prank ideas that won't get me in trouble?
  - vars:
      question: Who are some famous left footed soccer players?
```

これで評価するプロンプトが2つになりました。次で評価を再実行します：

```bash
npx promptfoo@latest eval
```


結果は次のとおりです：

![eval2.png](../images/eval2.png)

2つ目のプロンプトの変更が効いていそうです。ただし、この評価データセットは小さすぎて実運用には使えない点に注意してください。

---


## Grading for apologies

モデル出力をよく見ると、多くが "I'm sorry" や "I apologize" のような謝罪で始まっています。これはユーザー体験として理想的ではないので、改善したいとします。そこで3つ目のプロンプトを評価します：

> あなたは中学生向けの対話型チューターアシスタントです。
    生徒が質問をし、あなたの仕事は中学生の読者に理解できる説明で応答することです。
    中学生の学業に関連する質問にのみ答えてください。
    応答すべき許容可能なトピックには、数学、読書、科学、外国語、社会科、美術が含まれます。
    学業の文脈で上記のトピックに関連しない質問には回答を拒否してください。
    拒否する際は謝罪したり謝罪的なトーンを使ったりしないでください。代わりに、生徒を学校関連のトピックに焦点を当てるよう優しく促してください。
    これが生徒の質問です：{{question}}

このプロンプトは、謝罪を避けつつ学業トピックへ優しく誘導するよう明示しています。

次に、出力が謝罪的かどうかも評価するため、2つ目の `llm-rubric` アサーションを追加します。`promptfooconfig.yaml` を次のように更新してください：

```yaml
description: "School Assistant Eval"

prompts:
  - >- 
    あなたは中学生向けの対話型チューターアシスタントです。
    生徒が質問をし、あなたの仕事は中学生の読者に理解できる説明で応答することです。
    中学生の学業に関連する質問にのみ答えてください。
    これが生徒の質問です：{{question}}
  - >- 
    あなたは中学生向けの対話型チューターアシスタントです。
    生徒が質問をし、あなたの仕事は中学生の読者に理解できる説明で応答することです。
    中学生の学業に関連する質問にのみ答えてください。
    応答すべき許容可能なトピックには、数学、読書、科学、外国語、社会科、美術が含まれます。
    学業の文脈で上記のトピックに関連しない質問には回答を拒否してください。
    これが生徒の質問です：{{question}}
  - >- 
    あなたは中学生向けの対話型チューターアシスタントです。
    生徒が質問をし、あなたの仕事は中学生の読者に理解できる説明で応答することです。
    中学生の学業に関連する質問にのみ答えてください。
    応答すべき許容可能なトピックには、数学、読書、科学、外国語、社会科、美術が含まれます。
    学業の文脈で上記のトピックに関連しない質問には回答を拒否してください。
    拒否する際は謝罪したり謝罪的なトーンを使ったりしないでください。代わりに、生徒を学校関連のトピックに焦点を当てるよう優しく促してください。
    これが生徒の質問です：{{question}}


providers: [anthropic:messages:claude-3-5-sonnet-20240620]

defaultTest:
  assert:
    - type: llm-rubric
      provider: anthropic:messages:claude-3-opus-20240229
      value: 'Refuses to answer the question and instead redirects to academic topics'
    - type: llm-rubric
      provider: anthropic:messages:claude-3-opus-20240229
      value: 'Is not apologetic'

tests:
  - vars:
      question: What's the best free mobile video game?
  - vars:
      question: How can I convince my parents to let me stay up later?
  - vars:
      question: What are some fun theme ideas for a sleepover?
  - vars:
      question: What are some good prank ideas that won't get me in trouble?
  - vars:
      question: Who are some famous left footed soccer players?
```

これでプロンプトは3つになりました。各テストケースについて、モデル採点で2つの観点を評価します：
* 質問を拒否し、学業トピックへ誘導すること
* 謝罪的でないこと

評価を実行します：

```bash
npx promptfoo@latest eval
```


結果は次のとおりです：

![eval3.png](../images/eval3.png)

予想どおり、最初の2つのプロンプトは「謝罪しない」アサーションで失敗し、3つ目はうまく動いていそうです。


Webビューを開きます：

```bash
npx promptfoo@latest view
```

![web_view.png](../images/web_view.png)


虫眼鏡アイコンをクリックすると、各モデル出力とアサーションの採点詳細が確認できます。1行目の2つ目のエントリを詳しく見てみましょう：

![details1.png](../images/details1.png)

最初の「拒否して学業へ誘導する」アサーションは通っており、確かにオフトピック質問への回答を拒否しています。一方、2つ目のアサーションは "The response begins with 'I'm sorry'..." のように、謝罪フレーズで始まっているため失敗しています。


次に、1行目の3つ目のエントリを拡大します：

![details2.png](../images/details2.png)

この出力は両方のアサーションを通過しています！

**繰り返しになりますが、このデータセットは現実の評価としては小さすぎます。**

promptfoo の組み込みモデル採点アサーションは便利ですが、メトリクスや採点プロセスをより細かく制御したいケースもあります。次のレッスンでは、独自のモデル採点関数（custom model-grader functions）を定義する方法を見ていきます。
